# Optimal Nitrogen Rate

**Dataset description**: Fertilized corn crop in Nebraska.

Corn Trial conducted jointly with Dennis Francis and Jim Schepers, USDA-ARS, University of Nebraska, Lincoln, NE.  Corn trial located near Shelton, NE. Harvested in October, 2003

Summary:  Using the maximum yield Nitrogen Fertilization Optimization Algorithm (NFOA), grain yields were increased by 4 bu/ac when compared to the 80-85 N split (treatment 10 versus 4).  For treatment 10 and treatment 4, 80 lb of N /ac was applied at a fixed rate preplant.  When topdress N was applied on June 17, 2003, fixed rates were applied in treatments 2, 3, and 4.  Treatments 7-11 received topdress N applied using the variable N rate applicator (illustrated below) capable of sensing and treating each corn plant individually.  Grain yields were increased by 4 bu/ac for treatment 10 when compared to treatment 4, using 42 less lb N/ac.  As a result, the nitrogen use efficiencies of this treatment were 10% better than the split N treatment (treatment 10 vs 4, 40% NUE, and 30% NUE, respectively).

Source: <http://www.nue.okstate.edu/Nitrogen_Conference2003/Corn_Research.htm>

In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
# Fit polynomial: Example using yield responses to nitrogen rates
corn_yield_obs = [118,165,170,193,180,181,141,177,165,197,175]
nitrogen_rate =  [0,89,161,165,80,160,37,105,69,123,141]


In [3]:
f = figure(width=400, height=300)
f.scatter(nitrogen_rate,corn_yield_obs, size=5)
f.xaxis.axis_label = 'Nitrogen rate (kg/ha)'
f.yaxis.axis_label = 'Yield rate (kg/ha)'
show(f)

In [4]:
par = np.polyfit(nitrogen_rate, corn_yield_obs, 2)
print(par)

#par = np.polyfit(nitrogen_rate, obs_corn_yield, 2, full=True)
# When full=False (the default) just the coefficients are returned.
# When full=True diagnostic information is also provided.


[-3.41221057e-03  9.61329119e-01  1.15569115e+02]


In [5]:
# Create object that we can call to run the polynomial with the fitted coefficients
polyfun = np.poly1d(par) # Create object
print(polyfun) # polynomial function


           2
-0.003412 x + 0.9613 x + 115.6


In [6]:
# Compute fitting error
corn_yield_pred = polyfun(nitrogen_rate);
rmse = np.sqrt(np.mean((corn_yield_obs - corn_yield_pred)**2))
print(round(rmse,1),'kg/ha')

8.4 kg/ha


In [7]:
# Compute fitted curve
nitrogen_rate_pred = np.arange(np.min(nitrogen_rate),np.max(nitrogen_rate),0.1)
corn_yield_pred = polyfun(nitrogen_rate_pred)

In [8]:
f = figure(width=400, height=300)
f.scatter(nitrogen_rate,corn_yield_obs, size=5, legend_label='Observations')
f.line(nitrogen_rate_pred,corn_yield_pred, line_color='red',line_dash='dashed',line_width=2,legend_label='Fit')
f.xaxis.axis_label = 'Nitrogen rate (kg/ha)'
f.yaxis.axis_label = 'Yield rate (kg/ha)'
f.legend.location = 'top_left'
show(f)

In [9]:
first_derivative = np.diff(corn_yield_pred,n=1) # One value shorter

f = figure(width=400, height=300)
f.line(nitrogen_rate_pred[:-1], first_derivative)
f.line(nitrogen_rate_pred[:-1], y=0, line_dash='dashed', line_color='red')
f.xaxis.axis_label = 'Nitrogen rate (kg/ha)'
show(f)


In [10]:
idx_lowest_derivative = np.argmin(np.abs(first_derivative))
opt_nitrogen_rate = nitrogen_rate_pred[idx_lowest_derivative]
print(opt_nitrogen_rate )

140.8


In [11]:
from bokeh.models import Arrow, VeeHead

f = figure(width=400, height=300)
f.scatter(nitrogen_rate,corn_yield_obs, size=5, legend_label='Observations')
f.line(nitrogen_rate_pred,corn_yield_pred, line_color='red',line_dash='dashed',line_width=2,legend_label='Fit')
f.add_layout(Arrow(end=VeeHead(size=15),
                   x_start=opt_nitrogen_rate, 
                   y_start=np.max(corn_yield_pred), 
                   x_end=opt_nitrogen_rate, 
                   y_end=np.min(corn_yield_obs)-5))

f.xaxis.axis_label = 'Nitrogen rate (kg/ha)'
f.yaxis.axis_label = 'Yield rate (kg/ha)'
f.legend.location = 'top_left'
show(f)

